<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Libraries" data-toc-modified-id="Libraries-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Modelling" data-toc-modified-id="Modelling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modelling</a></span><ul class="toc-item"><li><span><a href="#Training" data-toc-modified-id="Training-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></li></ul></div>

# Introduction

This notebook trains an ALS model on the MovieLens 20M Dataset.

# Setup

In [1]:
%%capture
%cd ..

# Libraries

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.mllib.evaluation import RankingMetrics
import pyspark.sql.functions as f

In [3]:
from src import data

# Data

In [4]:
df = data.get_data("/tmp/ml-20m/ratings.csv")
df = data.add_label(df).cache()

train_df, valid_df, test_df = data.timesplit_data(df, train_dates=("2012-01-01", "2014-11-01"))

In [5]:
df.show(5)

+------+-------+------+-------------------+----------+
|userId|movieId|rating|          timestamp|adj_rating|
+------+-------+------+-------------------+----------+
|     1|      2|   3.5|2005-04-02 23:53:47|       1.0|
|     1|     29|   3.5|2005-04-02 23:31:16|       1.0|
|     1|     32|   3.5|2005-04-02 23:33:39|       1.0|
|     1|     47|   3.5|2005-04-02 23:32:07|       1.0|
|     1|     50|   3.5|2005-04-02 23:29:40|       1.0|
+------+-------+------+-------------------+----------+
only showing top 5 rows



Check date ranges in the train, valid and test splits:

In [6]:
for df_split in [train_df, valid_df, test_df]:
    df_split.selectExpr("count(timestamp)", "min(timestamp)", "max(timestamp)").show()

+----------------+-------------------+-------------------+
|count(timestamp)|     min(timestamp)|     max(timestamp)|
+----------------+-------------------+-------------------+
|         1734812|2012-01-01 00:00:40|2014-10-31 23:59:26|
+----------------+-------------------+-------------------+

+----------------+-------------------+-------------------+
|count(timestamp)|     min(timestamp)|     max(timestamp)|
+----------------+-------------------+-------------------+
|           79148|2014-11-01 00:01:36|2014-11-30 23:57:27|
+----------------+-------------------+-------------------+

+----------------+-------------------+-------------------+
|count(timestamp)|     min(timestamp)|     max(timestamp)|
+----------------+-------------------+-------------------+
|           79644|2014-12-01 00:05:52|2014-12-31 23:49:53|
+----------------+-------------------+-------------------+



# Modelling

## Training

Specify model hyperparameters:

In [7]:
rank = 400
maxIter = 20
coldStartStrategy = "drop" # no difference if switch to "nan"!
implicitPrefs = False

Instantiate the model:

In [8]:
als = ALS(rank=rank, 
          maxIter=maxIter,  
          coldStartStrategy=coldStartStrategy, 
          implicitPrefs=implicitPrefs,
          userCol="userId", 
          itemCol="movieId", 
          ratingCol="adj_rating")

Train the model:

In [9]:
%%time
model = als.fit(train_df)

Wall time: 9min 28s


## Evaluation

Evaluate the model on the validation set using the NDCG metric:

In [10]:
recsize = 10

In [11]:
valid_df.show(5)

+------+-------+------+-------------------+----------+
|userId|movieId|rating|          timestamp|adj_rating|
+------+-------+------+-------------------+----------+
|    96|    367|   2.0|2014-11-24 12:27:46|      -0.5|
|    96|    527|   4.0|2014-11-24 12:26:45|       1.5|
|    96|    608|   3.0|2014-11-24 12:27:26|       0.5|
|    96|   1270|   4.5|2014-11-24 12:27:32|       2.0|
|    96|   2011|   4.0|2014-11-24 12:28:24|       1.5|
+------+-------+------+-------------------+----------+
only showing top 5 rows



Build the predicted rankings for each user in the validation set:

In [12]:
valid_users = valid_df.select("userId").distinct()
predicted_ranks = model.recommendForUserSubset(valid_users, recsize)
# predicted_ranks.show(10)

We just need the list of movies for each user:

In [13]:
predicted_ranks = predicted_ranks \
    .rdd \
    .map(lambda r: (r.userId, [row.movieId for row in r.recommendations])) \
    .toDF(["userId", "predicted_ranking"])

Build the ground truth rankings:

In [14]:
actual_ranks = data.create_ground_truth_rankings(valid_df)

Check how bad is the cold start problem for new users:

In [15]:
# actual_ranks \
#     .join(predicted_ranks, "userId", "left") \
#     .withColumn("missing_recs", f.isnull("predicted_ranking").cast("int")) \
#     .selectExpr("count(missing_recs) AS total_users",
#                 "sum(missing_recs) AS new_users",
#                 "round(mean(missing_recs), 4) AS frac_users_with_missing_recs") \
#     .show(5)

Compute the metrics:

In [16]:
prediction_and_labels = predicted_ranks \
    .join(actual_ranks, "userId", "inner") \
    .drop("userId") \
    .rdd

metrics = RankingMetrics(prediction_and_labels)

metrics.ndcgAt(recsize)

0.0017735937566771254

In [17]:
metrics.precisionAt(recsize)

0.0013687600644122382